#Feeding models with realtime streaming data

In this example, we'll construct a very simple model of the number of posts relating to a given topic on twitter timelines around the world. We'll feed the model with live streaming data from the twitter API and run the model in real time.

##Ingredients

#### Libraries
In addition to our standard data analytics stack, we'll take advantage of the [tweepy](http://tweepy.readthedocs.org/) library for interacting with the twitter API, the standard python library for dealing with [JSON](https://docs.python.org/2/library/json.html) objects, and the standard python [threading](https://docs.python.org/2/library/threading.html) library, to give us access to both the stream and the plots in real time.

In [1]:
%pylab
import tweepy
import json
import pysd
import threading
from matplotlib import animation

Using matplotlib backend: MacOSX
Populating the interactive namespace from numpy and matplotlib


#### Twitter Credentials
If you want to execute this recipe, you'll need to [sign up for a twitter developer account](https://dev.twitter.com/) and create a file containing your credentials. 

Name this file `_twitter_credentials.py` and save it in the working directory. It should contain something similar to:

    consumer_key = 'sdjfhkdgjhsk8u09wejne4vdj8j34'
    consumer_secret = 'nvjsdv8wp43nneri'
    access_token = 'vndisoerihosfdbuttonmashingjkfdlasnvei'
    access_token_secret = 'navdjewrjodfjkmorebuttonmashingnjkasdoinv'

We can load these into our environment using the `import *` syntax.

In [2]:
from _twitter_credentials import *

#### Model

Our model is a simple delay process. The inflow of the process is the rate at which twitter users post messages containing our keywords, and the level decays over time as posts fall out of the top of users timelines.

We'll explicitly set the timescale in this demo, to show the behavior of the system in a short timeperiod. 

In [3]:
model = pysd.read_vensim('../../models/Twitter/Twitter.mdl')
model.set_components({'displacement_timescale':600})

## The Recipe


#### Listening to twitter
The first thing we'll do is to create a variable that will track the total number of posts recieved in the last model interation timeperiod. This counter will be reset after every model timestep.

In [4]:
counter = 0

Next, we'll construct a function that will be run whenever a tweet is recieved from the API. This function will increase the counter value, and format the tweet to print to the screen.

In [6]:
class TweetListener(tweepy.StreamListener):
    def on_data(self, data):
        global counter
        counter += 1
        
        # Twitter returns data in JSON format - we need to decode it first
        decoded = json.loads(data)

        # Also, we convert UTF-8 to ASCII ignoring all bad characters sent by users
        print '@%s: %s\n' % (decoded['user']['screen_name'], decoded['text'].encode('ascii', 'ignore'))
        return True

    def on_error(self, status):
        print status

The tweepy library manages formatting our credentials for the API request:

In [7]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

Finally we create the object that will parse the twitter stream, and start it within its own thread.

In [8]:
stream = tweepy.Stream(auth, TweetListener())

t = threading.Thread(target=stream.filter, kwargs={'track':['ISDC', 'PySD', 'ISDC15', 'Trump']})
t.daemon = True
t.start()

#### Animation

First we create a function that will be called at every step in the integration:

In [9]:
#make the animation
def animate(t):
    global counter
    #run the simulation forward
    time = model.components.t+dt
    model.run({'tweeting':counter}, 
              return_timestamps=time,
              return_columns=['tweeting', 'posts_on_timeline'],
              initial_condition='current',
              collect=True)
    out = model.get_record()
    ax.plot(out['tweeting'], 'r', label='Tweeting')
    ax.plot(out['posts_on_timeline'], 'b', label='Posts on Timeline')
    counter = 0

@TrumpIssues: RT @realDonaldTrump: "@DillyDoesIt Trump has no problem telling it how it is, isn't that what everyone has been wanting from a President? #

@FeedAtheism: Trump vs Trump http://t.co/mXXZDulCLP #atheist #atheism

@Cutiepi2u: RT @kausmickey: Trump on CBS: 1) immigration immigration immigration 2) retreats on "heroes" without apologizing 3) 3rd party run "very doa

@spikepriggen: RT @SenatorReid: I ask each Republican running for President: name one meaningful difference between your immigration policy and Trump's im

@NuelDark: @edwardsnowjob @thr ofcourse yes...as long as Donald Trump keeps being insensitive



Lastly we set the parameters for the animation, set up the figure, reset the counter (which has been accumulating posts since we ran the first part of the code) and start the animation.

In [10]:
#set the animation parameters
fps=1
seconds=60*30
dt=1./fps    

#set up the plot
fig, ax = plt.subplots()
ax.set_xlim(0,seconds)
title = ax.set_title('Expected Twitter Messages on First Page of Feed')
ax.set_xlabel('Seconds')
ax.set_ylabel('Posts, Posts/second')
    
#reset the counter to start fresh.
counter=0    
    
# call the animator.
animation.FuncAnimation(fig, animate, repeat=False,
                        frames=seconds*fps, interval=1000./fps, 
                        blit=False)

@marcelkinder: RT @SkyNews: It's emerged Donald Trump avoided military service five times during the Vietnam War http://t.co/MDyuP2lcrS http://t.co/PAhBVr

@dc2net: Lindsey Graham to Trump: Stop being ajackass http://t.co/LeVTFxMGaS

@lowdownny: Jon Stewart: Donald Trump Has No Control Over "Projectile Vomit Of Dickishness" http://t.co/Ai4mGve512

@Zory_F: Se popularizan en Texas las piatas de Donald Trump http://t.co/jHEvAmJmzU

@all_sabrina: Trump owes Sanders an apology http://t.co/p8KrHw93YE

@cnewportlogan: RT @SenatorReid: I ask each Republican running for President: name one meaningful difference between your immigration policy and Trump's im

@TonyPajamas305: Coincidence? I think not. #MakeAmericaGreatAgain  https://t.co/FxSNY0qke0

@TUSK81: RT @SenatorReid: I ask each Republican running for President: name one meaningful difference between your immigration policy and Trump's im

@Fox5KRBK: Has Trump gone too far? http://t.co/xtAe7wiIdW

@BurgerVonStadt: RT @redsteeze: Fun! now